In [13]:
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [74]:
%load_ext autoreload
%autoreload 2
from helper import draw_dot

class Value:
    def __init__(self, data, _children=(), _op='', label='') -> None:
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label
    
    def __repr__(self) -> str:
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        out = Value(self.data + other.data, _children=(self, other), _op='+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        if not isinstance(other, Value):
            other = Value(other)
        out = Value(self.data * other.data, _children=(self, other), _op='*')
        def _backward():
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        return self * other
    
    def __neg__(self): 
        return -1.0 * self
    
    def exp(self): 
        out = Value(math.exp(self.data), _children=(self,), _op='exp')
        def _backward():
            self.grad += out.grad * out.data
        out._backward = _backward
        return out
    
    def __pow__(self, other): # self ** other
        if not (isinstance(other, int) or isinstance(other, float)):
            assert False
        out = Value(self.data ** other, _children=(self,), _op='**')
        def _backward():
            self.grad += out.grad * (other * (self.data ** (other - 1.0)))
        out._backward = _backward
        return out
    
    def __sub__(self, other):
        return self + other.__neg__()

    def __truediv__(self, other): # self/other
        return self * (other ** (-1.0))
    
    # def tanh(self): 
    #     ex = math.exp(2*self.data)
    #     out = Value((ex - 1)/(ex + 1), _children=(self,), _op='tanh')
    #     def _backward(): 
    #         self.grad += (1 - out.data ** 2) * out.grad
    #     out._backward = _backward
    #     return out

    def tanh(self):
        e = (2*self).exp()
        return (e - 1)/(e + 1)
    
    def backward(self):
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
            return
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
# inputs x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# weights w1, w1
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
b = Value(6.88137, label='b')

w1x1 = w1 * x1; w1x1.label = "w1*x1"
w2x2 = w2 * x2; w2x2.label = "w2*x2"
w1x1w2x2 = w1x1 + w2x2; w1x1w2x2.label = "w1*x1 + w2*x2"
z = w1x1w2x2 + b; z.label = "z"
# a = z.tanh(); a.label = "a"

e = (2*z).exp()
a = (e - 1)/(e + 1)

In [82]:
import random

class Neuron:
    def __init__(self, nin):
        self.weights = [Value(random.uniform(-1.0, 1.0)) for i in range(nin)]
        self.bias = Value(random.uniform(-1.0, 1.0))

    def __call__(self, x): # forward pass
        _sum = self.bias
        for xi, wi in zip(x, self.weights):
            _sum = _sum + (xi * wi)
        return _sum.tanh()

class Layer: 
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for i in range(nout)]

    def __call__(self, x): # x is nin
        out = [neuron(x) for neuron in self.neurons]
        return out[0] if len(out) == 1 else out

class MLP: 
    def __init__(self, nin, nouts):
        self.layers = []
        prev = nin
        for i in nouts:
            self.layers.append(Layer(prev, i))
            prev = i

    def __call__(self, x): # forward pass
        y = x
        for i in self.layers:
            y = i(y)
        return y
            

l = MLP(3, [4, 4, 1])
# xs = [
#     [2, 3, 1],
#     [1, -6, 2],
#     [0, -1, 4],
#     [0, -1, 1]
# ]
x = [1, 3, 4]
print(l(x))

Value(data=-0.668937786504842)


In [92]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0]
ypred = [l(x) for x in xs]
ypred

[Value(data=0.6157934932427658),
 Value(data=-0.6695012453374227),
 Value(data=-0.7335987738673829),
 Value(data=0.802195979312708)]

In [93]:

for i in range(100):
    ypred = [l(x) for x in xs]

    loss = Value(0.0)
    for y, yp in zip(ys, ypred):
        loss = loss + (yp - y) ** 2

    print(loss.data)
    loss.backward()

    alpha = 0.01

    for i, layer in enumerate(l.layers):
        for j, neuron in enumerate(layer.neurons):
            for k, weight in enumerate(neuron.weights):
                l.layers[i].neurons[j].weights[k] = weight - alpha * weight.grad;
            l.layers[i].neurons[j].bias = neuron.bias - alpha * neuron.bias.grad;

0.36694011055313147
0.344946493714067
0.3245814696678245
0.30571039072160955
0.28821217947423544
0.2719775741074356
0.25690760804934887
0.2429123031735468
0.2299095542926688
0.2178241826401788
0.2065871367851377
0.19613482063832632
0.18640852966459223
0.17735397799825375
0.16892090081305114
0.1610627180042923
0.15373624698324959
0.14690145413828878
0.1405212362460774
0.13456122477211058
0.1289896075332932
0.1237769635633299
0.11889610819386352
0.11432194632619117
0.11003133262134218
0.10600293789490232
0.1022171213911732
0.09865580885813774
0.09530237648068271
0.09214154078348664
0.08915925461234699
0.08634260926454894
0.08367974278155316
0.0811597543528869
0.07877262471693351
0.07650914238754819
0.0743608354879423
0.07231990893620974
0.07037918670019418
0.068532058822346
0.06677243290662369
0.06509468975800206
0.06349364286939226
0.061964501459464805
0.06050283677683687
0.059104551400361804
0.05776585128098388
0.05648322028715216
0.05525339703257326
0.05407335378172827
0.0529402772447

In [87]:
test = [1]
for i in test:
    i = 3
test

[1]

In [94]:
print(ys)
print(ypred)

[1.0, -1.0, -1.0, 1.0]
[Value(data=0.9259784734387891), Value(data=-0.9023769948141576), Value(data=-0.912883019867243), Value(data=0.9436745669158564)]
